# Praca domowa z ML numer 12

<a href="https://colab.research.google.com/github/tomczj/ML24_25/blob/main/NLP_attention/homework_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Przygotowanie słów

In [ ]:
import json
import numpy as np
import gensim.downloader as api
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from matplotlib.cm import get_cmap
from collections import defaultdict

In [12]:
with open("semantic_classes.json", "r") as f:
    semantic_classes = json.load(f)

#przykładowe słowa 
print("GENDER: ",semantic_classes.get("gender"))
print("COMPARATIVE: ",semantic_classes.get("comparative"))

GENDER:  [['king', 'queen'], ['man', 'woman'], ['prince', 'princess'], ['waiter', 'waitress'], ['actor', 'actress'], ['husband', 'wife'], ['father', 'mother']]
COMPARATIVE:  [['big', 'bigger'], ['small', 'smaller'], ['fast', 'faster'], ['slow', 'slower'], ['high', 'higher'], ['low', 'lower'], ['bright', 'brighter']]


## Embedding

Korzystam z dostępnego modelu wpomninanego w zadaniu - dokładniej GloVe.

In [ ]:
model_glove = api.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


array([-0.32307 , -0.87616 ,  0.21977 ,  0.25268 ,  0.22976 ,  0.7388  ,
       -0.37954 , -0.35307 , -0.84369 , -1.1113  , -0.30266 ,  0.33178 ,
       -0.25113 ,  0.30448 , -0.077491, -0.89815 ,  0.092496, -1.1407  ,
       -0.58324 ,  0.66869 , -0.23122 , -0.95855 ,  0.28262 , -0.078848,
        0.75315 ,  0.26584 ,  0.3422  , -0.33949 ,  0.95608 ,  0.065641,
        0.45747 ,  0.39835 ,  0.57965 ,  0.39267 , -0.21851 ,  0.58795 ,
       -0.55999 ,  0.63368 , -0.043983, -0.68731 , -0.37841 ,  0.38026 ,
        0.61641 , -0.88269 , -0.12346 , -0.37928 , -0.38318 ,  0.23868 ,
        0.6685  , -0.43321 , -0.11065 ,  0.081723,  1.1569  ,  0.78958 ,
       -0.21223 , -2.3211  , -0.67806 ,  0.44561 ,  0.65707 ,  0.1045  ,
        0.46217 ,  0.19912 ,  0.25802 ,  0.057194,  0.53443 , -0.43133 ,
       -0.34311 ,  0.59789 , -0.58417 ,  0.068995,  0.23944 , -0.85181 ,
        0.30379 , -0.34177 , -0.25746 , -0.031101, -0.16285 ,  0.45169 ,
       -0.91627 ,  0.64521 ,  0.73281 , -0.22752 , 

In [ ]:
#funkcja do pozyskania embeddingów
def get_embeddings(words_list, embedding):
    vectors = []
    labels = []
    for word in words_list:
        vec = embedding[word]
        if vec is not None:
            vectors.append(vec)
            labels.append(word)

    return np.array(vectors), labels

In [ ]:
#zapisanie wyrazów do odpowiedniego formatu

#chcemy mieć poprawnie zapisane ze słownika w inny sensowny datatype
all_words = set()
for pairs in semantic_classes.values():
    for a, b in pairs:
        all_words.add(a.lower())
        all_words.add(b.lower())

#zapisujemy wyrazy do listy
listed_words = list(all_words)

### Globalne PCA

In [ ]:
#dosatejmy embedding
all_vectors, all_labels = get_embeddings(listed_words, model_glove)

In [ ]:
#przypisanie każdemu wyrazowi kalsy, aby potem lepiej zwizualizować to na obrazkach <- czysto wizualne zastosowanie
classes = list(semantic_classes.keys())
num_classes = len(classes)
cmap = get_cmap("tab10")
#przypisujemy kolorki dla każdej klasy
class_colors = {cls: cmap(i % num_classes) for i, cls in enumerate(classes)}

#ponownie identyfikujemy klasę dla każdego słowa <- nie jest to najbardziej optymlane podejście
word_to_class = defaultdict(lambda: "unknown")
for cls, pairs in semantic_classes.items():
    for a, b in pairs:
        word_to_class[a.lower()] = cls
        word_to_class[b.lower()] = cls

#kolorki do wykresów
colors = [class_colors[word_to_class[word]] for word in all_labels]

In [ ]:
#pca i skalowanie wektorów
pca = PCA(n_components = 2)
scaler = StandardScaler()

data = scaler.fit_transform(all_vectors)
data_pca = pca.fit_transform(data)